In [97]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import applications
from keras import optimizers
from keras.preprocessing.image import load_img, img_to_array

In [46]:
def VGG_16(weights_path=None, include_top=True, input_shape=(3,150,150)):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(512,(3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    if include_top:
        model.add(Flatten())
        model.add(Dense(4096, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(4096, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(1000, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)

    return model


In [47]:
# model = VGG_16('vgg16_weights.h5')
# sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(optimizer=sgd, loss='categorical_crossentropy')
# out = model.predict(im)
# print np.argmax(out)

In [88]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
test_data_dir = 'data/test'

top_model_weights_path = 'first_try.h5'
pre_trained_weights_path = './pre_trained_models/vgg16.h5'

nb_train_samples = 856
nb_validation_samples = 50
no_test_samples = 42
epochs = 1
batch_size = 16
no_of_classes = 21

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [80]:
# model = VGG_16(weights_path='./pre_trained_models/vgg16.h5', include_top=False, input_shape=(3,150,150))

In [81]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1./ 255,
    shear_range=0.2,
    horizontal_flip=True
)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./ 255)

train_generator = train_datagen.flow_from_directory(
    directory=train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

validation_generator = test_datagen.flow_from_directory(
    directory=validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

test_generator = test_datagen.flow_from_directory(
    directory=test_data_dir,
    target_size=(img_width, img_height),
    batch_size=1,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 856 images belonging to 21 classes.
Found 50 images belonging to 21 classes.
Found 42 images belonging to 21 classes.


In [82]:
# build the VGG16 network
conv_base = applications.VGG16(weights='imagenet', 
                           include_top=False,
                           input_shape=(img_width, img_height, 3))
conv_base.trainable = False

In [95]:
# print(conv_base.summary())

In [90]:
# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=conv_base.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(no_of_classes, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning

# top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
model = Sequential()
model.add(conv_base)
model.add(top_model)

model.load_weights(top_model_weights_path)
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
# for layer in model.layers[:25]:
#     layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [91]:
model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)


model.save_weights('first_try.h5')

/home/ninja/.local/lib/python3.5/site-packages/ipykernel_launcher.py:6: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/home/ninja/.local/lib/python3.5/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=53, epochs=1, validation_steps=50)`
  


Epoch 1/1
53/53 [==============================] - ETA: 2s - loss: 2.7370 - accuracy: 0.16 - 203s 4s/step - loss: 2.7395 - accuracy: 0.1667 - val_loss: 2.8204 - val_accuracy: 0.0585


In [110]:
# import 
# im = cv2.resize(cv2.imread('cat.jpg'), (224, 224)).astype(np.float32)
# im = im.transpose((2,0,1))
# im = np.expand_dims(im, axis=0

img = load_img('test.png', target_size=(img_width, img_height))
x = img_to_array(img)
preds = model.predict(x[None, :, :, :])

In [114]:
import numpy as np
np.argmax(preds[0])

train_generatorator.labels

array([ 0,  0,  1,  1,  2,  2,  3,  3,  3,  3,  4,  4,  4,  5,  5,  6,  6,
        7,  8,  8,  9,  9, 10, 10, 11, 11, 11, 12, 12, 13, 14, 14, 15, 17,
       17, 17, 18, 18, 19, 19, 19, 20], dtype=int32)